In [40]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps
import requests
import time
import json
import imp

# Import API keys
temp_file = imp.load_source("gkey", "../Project1/gkey.py")

gkey = temp_file.gkey

del temp_file

api_key = '4c40af20838b5ef25e18bea36e5b841e0ec5138d'
c = Census(api_key, year = 2016)

In [41]:
# Run Census Search to retrieve data on St. Louis Zip Codes

census_data = c.acs5.get(("NAME", "B19013_001E", 
                          "B01003_001E", 
                          "B02008_001E", 
                          "B02009_001E", 
                          "B02011_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"), {"for": "zip code tabulation area: 63136, 63147, 63115, 63120," +\
                                                                             "63107, 63113, 63106, 63137,"+\
                                                                             "63125, 63111, 63123, 63118,"+\
                                                                             "63112, 63119, 63116, 63143,"+\
                                                                             "63104, 63109, 63139, 63103,"+\
                                                                             "63101, 63110, 63102, 63117,"+\
                                                                             "63133, 63108, 63130, 63105"})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B02008_001E": "White", 
                                      "B02009_001E": "Black",
                                      "B02011_001E": "Asian",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "zip code tabulation area": "Zip Code"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in % of White (Employment Count / Population)
census_pd["White %"] = 100 * \
    census_pd["White"].astype(
        int) / census_pd["Population"].astype(int)

# Add in % of Black (Employment Count / Population)
census_pd["Black %"] = 100 * \
    census_pd["Black"].astype(
        int) / census_pd["Population"].astype(int)

# Add in % of Asian (Employment Count / Population)
census_pd["Asian %"] = 100 * \
    census_pd["Asian"].astype(
        int) / census_pd["Population"].astype(int)


# Final DataFrame
census_pd = census_pd[["Zip Code", "Name", "Population", "Household Income", "Per Capita Income", "Poverty Count", "Poverty Rate",
                       "Unemployment Rate", "White", "Black", "Asian", "White %", "Black %", "Asian %"
                          ]]
census_pd

,Zip Code,Name,Population,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate,White,Black,Asian,White %,Black %,Asian %
0,63101,ZCTA5 63101,3303.0,54442.0,44409.0,781.0,23.645171,5.237663,1306.0,1853.0,164.0,39.539812,56.100515,4.965183
1,63102,ZCTA5 63102,2314.0,44089.0,26807.0,116.0,5.012965,2.679343,1062.0,1081.0,136.0,45.894555,46.715644,5.877269
2,63103,ZCTA5 63103,7265.0,35968.0,36833.0,1457.0,20.055058,3.578802,3467.0,3324.0,438.0,47.721955,45.753613,6.028906
3,63104,ZCTA5 63104,20320.0,47816.0,32508.0,5150.0,25.344488,4.409449,10432.0,9616.0,354.0,51.338583,47.322835,1.742126
4,63105,ZCTA5 63105,17599.0,89750.0,58068.0,1285.0,7.301551,2.210353,14063.0,1459.0,2403.0,79.907949,8.290244,13.654185
5,63106,ZCTA5 63106,11989.0,14087.0,10183.0,6577.0,54.858620,8.007340,554.0,11482.0,11.0,4.620902,95.771124,0.091751
6,63107,ZCTA5 63107,10437.0,19685.0,12597.0,4557.0,43.661972,8.374054,1313.0,9166.0,24.0,12.580243,87.822171,0.229951
7,63108,ZCTA5 63108,21345.0,39162.0,37387.0,5799.0,27.167955,4.928555,11550.0,7684.0,2156.0,54.111033,35.999063,10.100726
8,63109,ZCTA5 63109,27072.0,54475.0,37509.0,1975.0,7.295361,2.323434,24307.0,2348.0,371.0,89.786495,8.673168,1.370420
9,63110,ZCTA5 63110,17423.0,45363.0,30782.0,3145.0,18.050852,4.023417,10349.0,6667.0,655.0,59.398496,38.265511,3.759398


In [42]:
# Create lists for append
Name = []
Address = []
Zip_Code = []
Lat = []
Lng = []

In [43]:
# Find Grocery Stores in St. Louis by searching Google Places

# Build URL
params = {"key": gkey,
          "radius": 50000,
         "location": "38.627003, -90.199402",
         "keyword": "grocery",  
             }

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

stl_grocery = requests.get(base_url, params=params)

stl_grocery = stl_grocery.json()

    
    
    
# Call and Append Data from API
for x in range(0,20):
    
        Name.append(stl_grocery["results"][x]["name"])
        Address.append(stl_grocery["results"][x]["vicinity"])
        Lat.append(stl_grocery["results"][x]["geometry"]["location"]["lat"])
        Lng.append(stl_grocery["results"][x]["geometry"]["location"]["lng"])
        
# Capture PageToken for Next Page        
        ptoken = stl_grocery["next_page_token"]

In [45]:
# Page 2
params = {"key": gkey,
          "radius": 50000,
         "location": "38.627003, -90.199402",
         "keyword": "grocery",
        "pagetoken": f"{ptoken}"
             }

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

stl_grocery = requests.get(base_url, params=params)

stl_grocery = stl_grocery.json()

    
    
    

for x in range(0,20):
    
        Name.append(stl_grocery["results"][x]["name"])
        Address.append(stl_grocery["results"][x]["vicinity"])
        Lat.append(stl_grocery["results"][x]["geometry"]["location"]["lat"])
        Lng.append(stl_grocery["results"][x]["geometry"]["location"]["lng"])
        
        ptoken = stl_grocery["next_page_token"]

In [46]:
# Page 3
params = {"key": gkey,
          "radius": 50000,
         "location": "38.627003, -90.199402",
         "keyword": "grocery",
        "pagetoken": f"{ptoken}"
             }

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

stl_grocery = requests.get(base_url, params=params)

stl_grocery = stl_grocery.json()

    
    
    

for x in range(0,20):
    
        Name.append(stl_grocery["results"][x]["name"])
        Address.append(stl_grocery["results"][x]["vicinity"])
        Lat.append(stl_grocery["results"][x]["geometry"]["location"]["lat"])
        Lng.append(stl_grocery["results"][x]["geometry"]["location"]["lng"])

In [47]:
# Create DataFrame for Google Data
dic = {
    "Name":Name,
    "Address":Address,
    "Lat":Lat,
    "Lng":Lng
}
supermarket_df = pd.DataFrame(dic)
supermarket_df.head(10)

,Name,Address,Lat,Lng
0,Fields Foods,"1500 Lafayette Ave, St. Louis",38.612256,-90.209282
1,Schnucks Arsenal,"5055 Arsenal, St. Louis",38.606803,-90.273927
2,Schnucks Lindell,"4171 Lindell Blvd, St. Louis",38.641805,-90.247256
3,Straub's,"8282 Forsyth Blvd, Clayton",38.652284,-90.344205
4,Schnucks Richmond Ctr.,"SCHNUCKS RICHMOND CTR., 6600 Clayton Rd, St. L...",38.633430,-90.314668
5,Dierbergs Markets - Brentwood Pointe,"8450 Eager Rd, St. Louis",38.627033,-90.339214
6,Schnucks South City,"3430 S Grand Blvd, St. Louis",38.595010,-90.242515
7,Schnucks Culinaria,"315 N 9th St, St. Louis",38.629126,-90.193581
8,Save-A-Lot,"1631 S Jefferson Ave, St. Louis",38.616046,-90.221852
9,Whole Foods Market,"4577 W Pine Blvd, St. Louis",38.641924,-90.261509


In [48]:
# Add Zip Code
supermarket_df['Zip Code'] = ""
supermarket_df.head()

,Name,Address,Lat,Lng,Zip Code
0,Fields Foods,"1500 Lafayette Ave, St. Louis",38.612256,-90.209282,
1,Schnucks Arsenal,"5055 Arsenal, St. Louis",38.606803,-90.273927,
2,Schnucks Lindell,"4171 Lindell Blvd, St. Louis",38.641805,-90.247256,
3,Straub's,"8282 Forsyth Blvd, Clayton",38.652284,-90.344205,
4,Schnucks Richmond Ctr.,"SCHNUCKS RICHMOND CTR., 6600 Clayton Rd, St. L...",38.633430,-90.314668,


In [49]:
# Build URL to Call Google Maps for Zip Codes
params = {"key": gkey}

# Loop through each store location
for index, row in supermarket_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    lat = row["Lat"]
    lng = row["Lng"]

    params["latlng"] = f"{lat},{lng}"

    # make request
    supermarket_zip = requests.get(base_url, params=params)
    
    # convert to json
    supermarket_zip = supermarket_zip.json()
    
# Pull Zip Codes

    supermarket_df.loc[index, "Zip Code"] = supermarket_zip["results"][0]["address_components"][6]["long_name"]
    if supermarket_df.loc[index, "Zip Code"] == "United States":
        supermarket_df.loc[index, "Zip Code"] = supermarket_zip["results"][0]["address_components"][7]["long_name"]
    elif supermarket_df.loc[index, "Zip Code"] == "Missouri":
        supermarket_df.loc[index, "Zip Code"] = supermarket_zip["results"][0]["address_components"][8]["long_name"]


supermarket_df.head(10)

,Name,Address,Lat,Lng,Zip Code
0,Fields Foods,"1500 Lafayette Ave, St. Louis",38.612256,-90.209282,63104
1,Schnucks Arsenal,"5055 Arsenal, St. Louis",38.606803,-90.273927,63139
2,Schnucks Lindell,"4171 Lindell Blvd, St. Louis",38.641805,-90.247256,63108
3,Straub's,"8282 Forsyth Blvd, Clayton",38.652284,-90.344205,63105
4,Schnucks Richmond Ctr.,"SCHNUCKS RICHMOND CTR., 6600 Clayton Rd, St. L...",38.633430,-90.314668,63117
5,Dierbergs Markets - Brentwood Pointe,"8450 Eager Rd, St. Louis",38.627033,-90.339214,63144
6,Schnucks South City,"3430 S Grand Blvd, St. Louis",38.595010,-90.242515,63118
7,Schnucks Culinaria,"315 N 9th St, St. Louis",38.629126,-90.193581,63101
8,Save-A-Lot,"1631 S Jefferson Ave, St. Louis",38.616046,-90.221852,63104
9,Whole Foods Market,"4577 W Pine Blvd, St. Louis",38.641924,-90.261509,63108


In [52]:
supermarket_df.to_csv("grocery.csv", index=False, header=True)

In [53]:
# Group by Zip Code to see how many stores per Zip Code
zip_group = supermarket_df.groupby(["Zip Code"])


store_count = zip_group['Name'].count()

zip_number = pd.DataFrame({"Grocery Store Count": store_count})

zip_number = zip_number.sort_values(["Grocery Store Count"], ascending=False)

zip_number.head()

,Grocery Store Count
Zip Code,
63141,5
63017,4
63118,4
63108,4
63116,3


In [54]:
# Merge Store Count with Census Data
df_merge = pd.merge(
    zip_number, census_pd, how="right", on=["Zip Code", "Zip Code"])
df_merge.head()

,Zip Code,Grocery Store Count,Name,Population,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate,White,Black,Asian,White %,Black %,Asian %
0,63118,4.0,ZCTA5 63118,28810.0,29880.0,19201.0,9996.0,34.696286,6.626171,12177.0,15464.0,1031.0,42.266574,53.675807,3.578619
1,63108,4.0,ZCTA5 63108,21345.0,39162.0,37387.0,5799.0,27.167955,4.928555,11550.0,7684.0,2156.0,54.111033,35.999063,10.100726
2,63116,3.0,ZCTA5 63116,44175.0,41273.0,24673.0,9304.0,21.061686,4.531975,30977.0,9475.0,3756.0,70.123373,21.448783,8.502547
3,63104,3.0,ZCTA5 63104,20320.0,47816.0,32508.0,5150.0,25.344488,4.409449,10432.0,9616.0,354.0,51.338583,47.322835,1.742126
4,63130,2.0,ZCTA5 63130,29488.0,63382.0,42794.0,4433.0,15.033234,4.086408,16619.0,11621.0,1561.0,56.358519,39.409251,5.293679


In [55]:
# Read in Overweight & Obesity CSV
overweight_df = pd.read_csv(
    "Rates of Overweight and Obesity by ZIP Code, City of St. Louis.csv", dtype="object", encoding="utf-8")
overweight_df.head()

,ZIP Code,Obese %,Overweight %,Overweight or Obese %
0,63136,40.50%,35.40%,76.00%
1,63147,39.60%,35.70%,75.30%
2,63115,38.90%,35.10%,74.00%
3,63120,39.50%,34.20%,73.60%
4,63107,38.30%,34.40%,72.70%


In [56]:
# Merge in Overweight & Obesity Data
complete_df = pd.merge(
    df_merge, overweight_df, how="left", left_on = 'Zip Code', right_on = 'ZIP Code')

# Clean up DataFrame
del complete_df['ZIP Code']
del complete_df['Name']
complete_df.fillna(0)

,Zip Code,Grocery Store Count,Population,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate,White,Black,Asian,White %,Black %,Asian %,Obese %,Overweight %,Overweight or Obese %
0,63118,4.0,28810.0,29880.0,19201.0,9996.0,34.696286,6.626171,12177.0,15464.0,1031.0,42.266574,53.675807,3.578619,27.80%,33.90%,61.70%
1,63108,4.0,21345.0,39162.0,37387.0,5799.0,27.167955,4.928555,11550.0,7684.0,2156.0,54.111033,35.999063,10.100726,16.90%,31.20%,48.10%
2,63116,3.0,44175.0,41273.0,24673.0,9304.0,21.061686,4.531975,30977.0,9475.0,3756.0,70.123373,21.448783,8.502547,25.20%,33.90%,59.00%
3,63104,3.0,20320.0,47816.0,32508.0,5150.0,25.344488,4.409449,10432.0,9616.0,354.0,51.338583,47.322835,1.742126,22.20%,35.20%,57.40%
4,63130,2.0,29488.0,63382.0,42794.0,4433.0,15.033234,4.086408,16619.0,11621.0,1561.0,56.358519,39.409251,5.293679,12.30%,29.10%,41.40%
5,63139,1.0,22072.0,49887.0,34433.0,2744.0,12.432041,2.328742,19222.0,2087.0,932.0,87.087713,9.455419,4.222544,22.40%,33.60%,56.00%
6,63133,1.0,7507.0,22632.0,12815.0,2900.0,38.630611,8.778473,617.0,7027.0,0.0,8.218996,93.605968,0.000000,0.00%,50.00%,50.00%
7,63119,1.0,33894.0,72214.0,40836.0,1869.0,5.514250,2.009205,30063.0,3099.0,1022.0,88.697115,9.143211,3.015283,31.30%,29.20%,60.40%
8,63117,1.0,9097.0,70680.0,51963.0,666.0,7.321095,2.781137,7514.0,1142.0,687.0,82.598659,12.553589,7.551940,17.70%,33.70%,51.30%
9,63113,1.0,11270.0,24283.0,16087.0,4156.0,36.876664,10.221828,373.0,10870.0,82.0,3.309672,96.450754,0.727595,38.50%,34.10%,72.60%


In [57]:
# Add Lat & Lng to Dataframe (for heatmap)
complete_df['Lat'] = ""
complete_df['Lng'] = ""

In [58]:
# Call Google Maps for Lat & Lng
params = {"key": gkey}

# Loop through each zip code
for index, row in complete_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    zip = row["Zip Code"]

    params["address"] = f"{zip}"

    # make request
    complete_loc = requests.get(base_url, params=params)
    
    # convert to json
    complete_loc = complete_loc.json()
    

# Add values to dataframe

    complete_df.loc[index, "Lat"] = complete_loc["results"][0]["geometry"]["location"]["lat"]
    complete_df.loc[index, "Lng"] = complete_loc["results"][0]["geometry"]["location"]["lng"]
    

In [59]:
complete_df.head()

,Zip Code,Grocery Store Count,Population,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate,White,Black,Asian,White %,Black %,Asian %,Obese %,Overweight %,Overweight or Obese %,Lat,Lng
0,63118,4.0,28810.0,29880.0,19201.0,9996.0,34.696286,6.626171,12177.0,15464.0,1031.0,42.266574,53.675807,3.578619,27.80%,33.90%,61.70%,38.5951,-90.2292
1,63108,4.0,21345.0,39162.0,37387.0,5799.0,27.167955,4.928555,11550.0,7684.0,2156.0,54.111033,35.999063,10.100726,16.90%,31.20%,48.10%,38.6475,-90.2573
2,63116,3.0,44175.0,41273.0,24673.0,9304.0,21.061686,4.531975,30977.0,9475.0,3756.0,70.123373,21.448783,8.502547,25.20%,33.90%,59.00%,38.5845,-90.2667
3,63104,3.0,20320.0,47816.0,32508.0,5150.0,25.344488,4.409449,10432.0,9616.0,354.0,51.338583,47.322835,1.742126,22.20%,35.20%,57.40%,38.616,-90.2292
4,63130,2.0,29488.0,63382.0,42794.0,4433.0,15.033234,4.086408,16619.0,11621.0,1561.0,56.358519,39.409251,5.293679,12.30%,29.10%,41.40%,38.6683,-90.3231


In [60]:
complete_df.to_csv("Data.csv", index=False, header=True)

In [62]:
# Same DataFrame with Clean Formatting
complete_df["Household Income"] = complete_df["Household Income"].astype(float).map("${:,.0f}".format)
complete_df["Per Capita Income"] = complete_df["Per Capita Income"].astype(float).map("${:,.0f}".format)
complete_df["Poverty Rate"] = complete_df["Poverty Rate"].astype(float).map("{0:.1f}%".format)
complete_df["Unemployment Rate"] = complete_df["Unemployment Rate"].astype(float).map("{0:.1f}%".format)
complete_df["White %"] = complete_df["White %"].astype(float).map("{0:.1f}%".format)
complete_df["Black %"] = complete_df["Black %"].astype(float).map("{0:.1f}%".format)
complete_df["Asian %"] = complete_df["Asian %"].astype(float).map("{0:.1f}%".format)

complete_df.head()

ValueError: could not convert string to float: '$29,880'

In [63]:
complete_df.to_csv("FoodDesertData.csv", index=False, header=True)

In [64]:
# Only Show St. Louis City Grocery Stores in Store Location DataFrame
grocery_stl = supermarket_df.loc[supermarket_df['Zip Code'].isin(['63118','63108','63116','63139','63130','63113','63117',
                                                                  '63111','63110','63109','63105','63101'])]
grocery_stl.head()

,Name,Address,Lat,Lng,Zip Code
1,Schnucks Arsenal,"5055 Arsenal, St. Louis",38.606803,-90.273927,63139
2,Schnucks Lindell,"4171 Lindell Blvd, St. Louis",38.641805,-90.247256,63108
3,Straub's,"8282 Forsyth Blvd, Clayton",38.652284,-90.344205,63105
4,Schnucks Richmond Ctr.,"SCHNUCKS RICHMOND CTR., 6600 Clayton Rd, St. L...",38.633430,-90.314668,63117
6,Schnucks South City,"3430 S Grand Blvd, St. Louis",38.595010,-90.242515,63118


In [65]:
grocery_stl.to_csv("grocery_stl.csv", index=False, header=True)